In [2]:
library(tidyverse)
library(devtools)
library("GENESPACE")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis

GENESPACE v1.3.1: synteny and orthology constrained comparative
        genomics



In [3]:
out <- readRDS("/scratch/jpm73279/16.GENESPACE/all_species.genespace_results/genespace.results.all_species.rds")

In [36]:
zm_c4_genes <- read_delim("/home/jpm73279/genome_downloads/C4_markers/Sb.extra_large.C4_genes.bed", col_names = c("chr", "start", "end", "gene_name", "marker", "ct"), delim = "\t") %>% 
    dplyr::mutate(genome = "Sb")

Rows: 31 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, gene_name, marker
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
# zm_ACRs_all. <- zm_ACRs_all %>% 
#     dplyr::filter(start < end)

In [37]:
qreturn_cts <- query_hits(gsParam = out, bed = zm_c4_genes, synOnly = TRUE)

In [38]:
flattened_list_zm_acrs <- bind_rows(qreturn_cts, .id = "list_name")

In [39]:
unique(flattened_list_zm_acrs$genome2)

[1] "Sb" "Pm" "Uf" "Zm" "Os"

In [60]:
isolated_zm_regions <- flattened_list_zm_acrs %>% 
    dplyr::filter(isAnchor == TRUE) %>% 
    #dplyr::filter(genome2 == "Os") %>% 
    dplyr::select(chr1, start1, end1, id1, chr2, start2, end2, id2, list_name, genome2)

In [61]:
glimpse(isolated_zm_regions)

Rows: 942
Columns: 10
$ chr1      <chr> "Chr01", "Chr01", "Chr01", "Chr01", "Chr01", "Chr01", "Chr01…
$ start1    <dbl> 5624141, 5624141, 5624141, 5624141, 5624141, 5645438, 564543…
$ end1      <dbl> 5644231, 5644231, 5644231, 5644231, 5644231, 5648604, 564860…
$ id1       <chr> "Sorbiv5.1_pg2504", "Sorbiv5.1_pg2504", "Sorbiv5.1_pg2504", …
$ chr2      <chr> "Chr01", "CM009691.2", "Chr09", "chr1", "Chr3", "Chr01", "CM…
$ start2    <dbl> 5624141, 53386051, 3627492, 289483624, 32096846, 5645438, 53…
$ end2      <dbl> 5644231, 53393086, 3635426, 289502480, 32104074, 5648604, 53…
$ id2       <chr> "Sorbiv5.1_pg2504", "RLN18085.1", "Urofu.9G069100", "Zm00001…
$ list_name <chr> "Sb, Chr01: 5644231-5716503", "Sb, Chr01: 5644231-5716503", …
$ genome2   <chr> "Sb", "Pm", "Uf", "Zm", "Os", "Sb", "Pm", "Pm", "Uf", "Zm", …


In [62]:
updated_zm_acr_list <- isolated_zm_regions %>% 
      group_by(list_name) %>%
  mutate(
    list_name = str_replace_all(list_name, "[, :;-]", "_"),
    list_name = paste(list_name, "_syntenic_regions", cur_group_id(), sep=""),
    region_name = paste("syntenic_regions", cur_group_id(), sep = "")
  ) %>%
  ungroup()


In [63]:
tail(updated_zm_acr_list)

chr1,start1,end1,id1,chr2,start2,end2,id2,list_name,genome2,region_name
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
Chr10,48938716,48939481,Sorbiv5.1_pg10298,Chr10,48938716,48939481,Sorbiv5.1_pg10298,Sb__Chr10__48816167_48945840_syntenic_regions31,Sb,syntenic_regions31
Chr10,48945840,48950740,Sobic.010G160900,Chr10,48945840,48950740,Sobic.010G160900,Sb__Chr10__48816167_48945840_syntenic_regions31,Sb,syntenic_regions31
Chr10,48945840,48950740,Sobic.010G160900,CM009698.2,38708725,38710921,RLN12610.1,Sb__Chr10__48816167_48945840_syntenic_regions31,Pm,syntenic_regions31
Chr10,48945840,48950740,Sobic.010G160900,Chr04,8681861,8684675,Urofu.4G116900,Sb__Chr10__48816167_48945840_syntenic_regions31,Uf,syntenic_regions31
Chr10,48945840,48950740,Sobic.010G160900,chr9,71278216,71280945,Zm00001eb383720,Sb__Chr10__48816167_48945840_syntenic_regions31,Zm,syntenic_regions31
Chr10,48945840,48950740,Sobic.010G160900,Chr6,20454249,20457005,LOC_Os06g35140,Sb__Chr10__48816167_48945840_syntenic_regions31,Os,syntenic_regions31


In [64]:
updated_zm_acr_list %>% 
    dplyr::filter(id1 == "Sorbiv5.1_pg4709")

chr1,start1,end1,id1,chr2,start2,end2,id2,list_name,genome2,region_name
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
Chr01,75300945,75306070,Sorbiv5.1_pg4709,Chr01,75300945,75306070,Sorbiv5.1_pg4709,Sb__Chr01__75268593_75329395_syntenic_regions5,Sb,syntenic_regions5
Chr01,75300945,75306070,Sorbiv5.1_pg4709,CM009690.2,7687203,7691961,RLN38740.1,Sb__Chr01__75268593_75329395_syntenic_regions5,Pm,syntenic_regions5
Chr01,75300945,75306070,Sorbiv5.1_pg4709,Chr09,39463750,39468524,Urofu.9G489800,Sb__Chr01__75268593_75329395_syntenic_regions5,Uf,syntenic_regions5
Chr01,75300945,75306070,Sorbiv5.1_pg4709,chr9,147964764,147969638,Zm00001eb398370,Sb__Chr01__75268593_75329395_syntenic_regions5,Zm,syntenic_regions5
Chr01,75300945,75306070,Sorbiv5.1_pg4709,Chr3,8218790,8224127,LOC_Os03g15050,Sb__Chr01__75268593_75329395_syntenic_regions5,Os,syntenic_regions5


In [65]:
updated_zm_acr_list %>% 
    dplyr::filter(region_name == "syntenic_regions5")

chr1,start1,end1,id1,chr2,start2,end2,id2,list_name,genome2,region_name
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
Chr01,75267709,75268593,Sobic.001G432550,Chr01,75267709,75268593,Sobic.001G432550,Sb__Chr01__75268593_75329395_syntenic_regions5,Sb,syntenic_regions5
Chr01,75270555,75271610,Sobic.001G432600,Chr01,75270555,75271610,Sobic.001G432600,Sb__Chr01__75268593_75329395_syntenic_regions5,Sb,syntenic_regions5
Chr01,75270555,75271610,Sobic.001G432600,CM009691.2,42675683,42676852,RLN15384.1,Sb__Chr01__75268593_75329395_syntenic_regions5,Pm,syntenic_regions5
Chr01,75270555,75271610,Sobic.001G432600,Chr09,39441717,39442700,Urofu.9G489500,Sb__Chr01__75268593_75329395_syntenic_regions5,Uf,syntenic_regions5
Chr01,75270555,75271610,Sobic.001G432600,chr1,35784439,35785125,Zm00001eb011120,Sb__Chr01__75268593_75329395_syntenic_regions5,Zm,syntenic_regions5
Chr01,75273372,75274455,Sobic.001G432700,Chr01,75273372,75274455,Sobic.001G432700,Sb__Chr01__75268593_75329395_syntenic_regions5,Sb,syntenic_regions5
Chr01,75273372,75274455,Sobic.001G432700,CM009690.2,7700031,7701286,RLN41872.1,Sb__Chr01__75268593_75329395_syntenic_regions5,Pm,syntenic_regions5
Chr01,75273372,75274455,Sobic.001G432700,Chr09,39446268,39447266,Urofu.9G489600,Sb__Chr01__75268593_75329395_syntenic_regions5,Uf,syntenic_regions5
Chr01,75273372,75274455,Sobic.001G432700,chr1,35722343,35723223,Zm00001eb011110,Sb__Chr01__75268593_75329395_syntenic_regions5,Zm,syntenic_regions5


In [58]:
combined_zm_markers_regions <- left_join(updated_zm_acr_list, zm_c4_genes, c("id1" = "gene_name"))

In [59]:
combined_zm_markers_regions %>% 
    dplyr::filter(is.na(marker) != TRUE) %>% 
    dplyr::group_by(marker) %>% 
    summarise(counts = n())

marker,counts
<chr>,<int>
ASP_AT1,6
ASP_AT2,5
AlaAT2,8
AlaAT3_ortho_count_1__AlaAT4_ortho_count_1__AlaAT3_ortho_count_2__AlaAT4_ortho_count_2,26
BASS,5
CA4,7
CA6_ortho_count_1,12
CA6_ortho_count_2,51
DIC,6


In [25]:
# List of species
species <- c('Zm', 'Pm', 'Sb', 'Os', 'Uf')

# Create a list to hold the data frames for each species
list_of_regions <- list()

# Iterate over each species
for (sp in species) {
    # Filter the data frame based on the species
    species_regions <- updated_zm_acr_list %>% 
        dplyr::filter(genome2 == sp)
    
    # Select regions from the filtered data frame
    species_select_regions <- species_regions %>% 
        dplyr::select(chr1, start1, end1, id1, region_name)
    
    list_of_regions[[sp]] <- species_select_regions
    
    # Extract and write the final regions for this species
    species_regions_final <- species_regions %>% 
        dplyr::select(chr2, start2, end2, id2, region_name)
    
    write_delim(species_regions_final, 
                file = paste0("/scratch/jpm73279/16.GENESPACE/all_species.genespace_results/generated_syntenic_blocks/", sp, ".syntenic_genes.all_zm_ACRs.bed"), 
                col_names = FALSE, delim = "\t")
    
    # Summarize and write the acr blocks for this species
    species_acr_blocks <- species_regions_final %>% 
        group_by(chr2, region_name) %>%
        dplyr::summarise(start = min(start2), end = max(end2)) %>% 
        dplyr::ungroup() %>% 
        dplyr::select(chr2, start, end, region_name)
    
    write_delim(species_acr_blocks, 
                file = paste0("/scratch/jpm73279/16.GENESPACE/all_species.genespace_results/generated_syntenic_blocks/", sp, ".syntenic_regions.zm_acrs.bed"), 
                col_names = FALSE, delim = "\t")
}

`summarise()` has grouped output by 'chr2'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'chr2'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'chr2'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'chr2'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'chr2'. You can override using the
`.groups` argument.
